#  Build a Neural network with Hyper-parameter fine tuning model

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.datasets import fashion_mnist
from keras_tuner import RandomSearch

In [2]:
Trails=2 
epochs=3 

# Get the fashion_MNIST dataset

In [3]:
(xtrain,ytrain),(xval,yval)=fashion_mnist.load_data()
xtrain=np.expand_dims(xtrain.astype('float32')/255.,-1)
xval=np.expand_dims(xval.astype('float32')/255.,-1)
ytrain=to_categorical(ytrain,10)
yval=to_categorical(yval,10)

In [4]:
def build_model(hp):
    """
    Function to build a TF model based on hyperparameter values
    
    Args:
    hp (hyperparameter): values
    
    Returns:
    Model: compiled model
    """
    num_layers = hp.Int('num_layers', 2, 8, default=6)
    lr = hp.Choice('learning_rate', [0.0001, 0.05])
    inputs = layers.Input(shape=(28, 28, 1))
    x = inputs
    for idx in range(num_layers):
        idx_str = str(idx)
        filters = hp.Int('filters_' + idx_str, 32, 256, step=32, default=64)
        x = layers.Conv2D(filters=filters, kernel_size=3, padding='same', activation='relu')(x)

        if x.shape[1] >= 8:
            pool_type = hp.Choice('pool_' + idx_str, values=['max', 'avg'])
            if pool_type == 'max':
                x = layers.MaxPooling2D(2)(x)
            elif pool_type == 'avg':
                x = layers.AveragePooling2D(2)(x)

    x = layers.Flatten()(x)
    outputs = layers.Dense(10, activation='softmax')(x)

    # Build model
    model = keras.Model(inputs, outputs)
    model.compile(optimizer=Adam(lr), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [5]:
tuner=RandomSearch(build_model,objective='val_accuracy',max_trials=Trails,
                  project_name='FASHION_MNIST_results')

Reloading Tuner from .\FASHION_MNIST_results\tuner0.json


In [7]:
tuner.search_space_summary()


Search space summary
Default search space size: 10
num_layers (Int)
{'default': 6, 'conditions': [], 'min_value': 2, 'max_value': 8, 'step': 1, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.0001, 'conditions': [], 'values': [0.0001, 0.05], 'ordered': True}
filters_0 (Int)
{'default': 64, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
pool_0 (Choice)
{'default': 'max', 'conditions': [], 'values': ['max', 'avg'], 'ordered': False}
filters_1 (Int)
{'default': 64, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
pool_1 (Choice)
{'default': 'max', 'conditions': [], 'values': ['max', 'avg'], 'ordered': False}
filters_2 (Int)
{'default': 64, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
filters_3 (Int)
{'default': 64, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
filters_4 (Int)
{'default': 64, 'conditions': [], 'min_value

In [8]:
best_model=tuner.get_best_models(num_models=1)[0]
best_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 96)        960       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 96)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 192)       166080    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 192)         0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 224)         387296

# Build an image classifier model with Pytorch

In [9]:
import torch
import torchvision
import torchvision.transforms as transforms

In [10]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

batch_size = 4
trainset = torchvision.datasets.SVHN(root='./data', split='train',
                                     download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.SVHN(root='./data', split='test',
                                    download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=1)

classes = tuple(str(i) for i in range(5))


 43%|██████████████████████████▌                                   | 78010277/182040794 [2:09:21<2:52:30, 10050.62it/s]


RuntimeError: File not found or corrupted.

In [13]:







import torch
from torch.utils.data import DataLoader, TensorDataset

# Example random data
train_data = torch.randn((100, 3, 32, 32))  # Assuming image data with shape (3, 32, 32)
train_labels = torch.randint(0, 10, (100,))

# Create a TensorDataset
train_dataset = TensorDataset(train_data, train_labels)

# Create a DataLoader
batch_size = 10
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Rest of your code using trainloader
dataiter = iter(trainloader)
images, labels = next(dataiter)
# ... rest of the code


In [14]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()


In [15]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


# Train The network


In [16]:
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')






Finished Training


In [31]:
PATH = './svhn_net.pth'
torch.save(net.state_dict(), PATH)

# Test the network on the test data

In [32]:


import torch
from torch.utils.data import DataLoader, TensorDataset

# Example random data
train_data = torch.randn((100, 3, 32, 32))  # Assuming image data with shape (3, 32, 32)
train_labels = torch.randint(0, 10, (100,))

# Create a TensorDataset
train_dataset = TensorDataset(train_data, train_labels)

# Create a DataLoader
batch_size = 10
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Rest of your code using trainloader
dataiter = iter(trainloader)
images, labels = next(dataiter)



In [23]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [24]:
outputs = net(images)

IndexError: list index out of range

In [33]:


import torch
from torch.utils.data import DataLoader, TensorDataset

# Example random data
train_data = torch.randn((100, 3, 32, 32))  # Assuming image data with shape (3, 32, 32)
train_labels = torch.randint(0, 10, (100,))

# Create a TensorDataset
train_dataset = TensorDataset(train_data, train_labels)

# Create a DataLoader
batch_size = 10
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Rest of your code using trainloader
dataiter = iter(trainloader)
images, labels = next(dataiter)


In [34]:


import torch
from torch.utils.data import DataLoader, TensorDataset

# Example random data
train_data = torch.randn((100, 3, 32, 32))  # Assuming image data with shape (3, 32, 32)
train_labels = torch.randint(0, 10, (100,))

# Create a TensorDataset
train_dataset = TensorDataset(train_data, train_labels)

# Create a DataLoader
batch_size = 10
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Rest of your code using trainloader
dataiter = iter(trainloader)
images, labels = next(dataiter)
